#0. Install PyOpenCL library
We need to install the library first. Also, if you run the code and it shows "Can't find library", then you need to rerun this step. Google Colab will delete every library installation everytime the runtime reset.

In [1]:
!pip install pyopencl

     |████████████████████████████████| 727kB 4.9MB/s 
     |████████████████████████████████| 61kB 8.6MB/s 
  Created wheel for pytools: filename=pytools-2020.1-py2.py3-none-any.whl size=59602 sha256=7bcedb5a10ed6ebbed8205cc8e3567db1d57e732911626ebf0f0b23a34253302
  Stored in directory: /root/.cache/pip/wheels/6f/da/1b/946775a88291378182ed92c9800d6d0ebc2a554cb89829cc24
Successfully built pytools


# 1. Setup Runtime
At the "Runtime" menu, choose "change runtime type". Then, make sure that you choose "Python 3" as runtime type, and "GPU" as hardware accelerator.

# 2. Check if OpenCL can run correctly
Run code in the following cell, it should show the information about platform(s) and device(s) on the current runtime.

In [0]:
import pyopencl as cl
platforms = cl.get_platforms()
platform_count = 0
for platform in platforms:
  print('Platform #', platform_count)
  print('\tVendor:', platform.get_info(cl.platform_info.VENDOR))
  print('\tName:', platform.get_info(cl.platform_info.NAME))
  print('\tProfile:', platform.get_info(cl.platform_info.PROFILE))
  devices = platform.get_devices(cl.device_type.ALL)
  device_count = 0
  for device in devices:
    print('\tDevice #', device_count)
    print('\t\tVendor', device.get_info(cl.device_info.VENDOR))
    print('\t\tName', device.get_info(cl.device_info.NAME))
    print('\t\tMax Compute Units', device.get_info(cl.device_info.MAX_COMPUTE_UNITS))
    print('\t\tMax Clock Frequency', device.get_info(cl.device_info.MAX_CLOCK_FREQUENCY))
    print('\t\tGlobal Mem Size', device.get_info(cl.device_info.GLOBAL_MEM_SIZE))
    print('\t\tLocal Mem Size', device.get_info(cl.device_info.LOCAL_MEM_SIZE))
    device_count = device_count + 1
  pltform_count = platform_count + 1

Platform # 0
	Vendor: NVIDIA Corporation
	Name: NVIDIA CUDA
	Profile: FULL_PROFILE
	Device # 0
		Vendor NVIDIA Corporation
		Name Tesla P4
		Max Compute Units 20
		Max Clock Frequency 1113
		Global Mem Size 7981694976
		Local Mem Size 49152


# 3. Example OpenCL code
This code perform calculating a + b, where a and b are 50,000 random float. The result is in rest_np. Please compare the python's OpenCL functions with C's OpenCL function in the slides. 

In [0]:
import numpy as np
import pyopencl as cl
a_np = np.random.rand(50000).astype(np.float32)
b_np = np.random.rand(50000).astype(np.float32)

ctx = cl.create_some_context()
queue = cl.CommandQueue(ctx)

mf = cl.mem_flags
a_g = cl.Buffer(ctx, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf=a_np)
b_g = cl.Buffer(ctx, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf=b_np)

prg = cl.Program(ctx, """
__kernel void sum(
    __global const float *a_g, __global const float *b_g, __global float *res_g)
{
  int gid = get_global_id(0);
  res_g[gid] = a_g[gid] + b_g[gid];
}
""").build()

res_g = cl.Buffer(ctx, mf.WRITE_ONLY, a_np.nbytes)
prg.sum(queue, a_np.shape, None, a_g, b_g, res_g)

res_np = np.empty_like(a_np)
cl.enqueue_copy(queue, res_np, res_g)

# Check on CPU with Numpy:
print('Calculate result = a + b')
print('a:', a_np)
print('b:', b_np)
print('result:', res_np)
print('Check the result (result - (a + b):', res_np - (a_np + b_np))
print('Find vector norm of the the previous step:', np.linalg.norm(res_np - (a_np + b_np)))
assert np.allclose(res_np, a_np + b_np)

Calculate result = a + b
a: [0.7442459  0.92191094 0.31668732 ... 0.64450383 0.6854204  0.95351774]
b: [0.2560115  0.3531105  0.7055     ... 0.42537183 0.8089764  0.5197493 ]
result: [1.0002574 1.2750214 1.0221874 ... 1.0698757 1.4943968 1.4732671]
Check the result (result - (a + b): [0. 0. 0. ... 0. 0. 0.]
Find vector norm of the the previous step: 0.0


# 4. Is it really faster?
Let's measure the execution time. We will use python's time function, it's not accurate but shold be enough.

In [0]:
import numpy as np
import pyopencl as cl
import time
a_np = np.random.rand(50000).astype(np.float32)
b_np = np.random.rand(50000).astype(np.float32)

def perform_sequential_addition(a_np, b_np):
  result = np.empty_like(a_np)
  size = a_np.size
  for x in range(0, size):
    result[x] = a_np[x] + b_np[x]
  return result

def perform_opencl_addition(a_np, b_np):
  ctx = cl.create_some_context()
  queue = cl.CommandQueue(ctx)

  mf = cl.mem_flags
  a_g = cl.Buffer(ctx, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf=a_np)
  b_g = cl.Buffer(ctx, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf=b_np)

  prg = cl.Program(ctx, """
  __kernel void sum(
      __global const float *a_g, __global const float *b_g, __global float *res_g)
  {
    int gid = get_global_id(0);
    res_g[gid] = a_g[gid] + b_g[gid];
  }
  """).build()

  res_g = cl.Buffer(ctx, mf.WRITE_ONLY, a_np.nbytes)
  prg.sum(queue, a_np.shape, None, a_g, b_g, res_g)

  res_np = np.empty_like(a_np)
  cl.enqueue_copy(queue, res_np, res_g)
  return res_np

# Run sequential code
start_time = time.time()
res_np = perform_sequential_addition(a_np, b_np)
end_time = time.time()

print('Find vector norm to check result:', np.linalg.norm(res_np - (a_np + b_np)))
print("Sequential processing time:", end_time - start_time)

# Run parallel code
start_time = time.time()
res_np = perform_opencl_addition(a_np, b_np)
end_time = time.time()

print('Find vector norm to check result:', np.linalg.norm(res_np - (a_np + b_np)))
print("Parallel processing time:", end_time - start_time)



Find vector norm to check result: 0.0
Sequential processing time: 0.02369976043701172
Find vector norm to check result: 0.0
Parallel processing time: 0.1260664463043213


#5. Wait, it seems like sequential code runs faster?
That's right, if your data is not big emough, the overhead of transfering data to/from GPU will overrun the parallel performance. So, let's increase the data size and compare again.

Note: this is not fair comparison, but for simplicity, this should be OK.

In [0]:
import numpy as np
import pyopencl as cl
import time
a_np = np.random.rand(5000000).astype(np.float32)
b_np = np.random.rand(5000000).astype(np.float32)

def perform_sequential_addition(a_np, b_np):
  result = np.empty_like(a_np)
  size = a_np.size
  for x in range(0, size):
    result[x] = a_np[x] + b_np[x]
  return result

def perform_opencl_addition(a_np, b_np):
  ctx = cl.create_some_context()
  queue = cl.CommandQueue(ctx)

  mf = cl.mem_flags
  a_g = cl.Buffer(ctx, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf=a_np)
  b_g = cl.Buffer(ctx, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf=b_np)

  prg = cl.Program(ctx, """
  __kernel void sum(
      __global const float *a_g, __global const float *b_g, __global float *res_g)
  {
    int gid = get_global_id(0);
    res_g[gid] = a_g[gid] + b_g[gid];
  }
  """).build()

  res_g = cl.Buffer(ctx, mf.WRITE_ONLY, a_np.nbytes)
  prg.sum(queue, a_np.shape, None, a_g, b_g, res_g)

  res_np = np.empty_like(a_np)
  cl.enqueue_copy(queue, res_np, res_g)
  return res_np

start_time = time.time()
res_np = perform_sequential_addition(a_np, b_np)
end_time = time.time()
sequential_time = end_time - start_time;

print('Find vector norm to check result:', np.linalg.norm(res_np - (a_np + b_np)))
print("Sequential processing time:", sequential_time)

start_time = time.time()
res_np = perform_opencl_addition(a_np, b_np)
end_time = time.time()
parallel_time = end_time - start_time

print('Find vector norm to check result:', np.linalg.norm(res_np - (a_np + b_np)))
print("Parallel processing time:", parallel_time)

print("Speed up:", sequential_time/parallel_time)

Find vector norm to check result: 0.0
Sequential processing time: 2.0483665466308594
Find vector norm to check result: 0.0
Parallel processing time: 0.1511242389678955
Speed up: 13.554189325419927


# Question #1
So, you can see that parallel code run faster. From the speed up value in the previous step and number of computational core in step 2. Can you guess the ratio of parallel portion? You can answer by edit this cell and put your answer below. Please also show your calculation steps.

Answer: from S = 1/1-P then move P to other side P = S-1/S
        then S = 13.554 then P = 0.9262 so 
        the ratio of parallel portion is 92 %  

# 6. Matrix Multiplication
Matrix multiplication is a very common operation on many fields, including simultation, image processing, data analytics. So, in short, we have two matrix, A and B, and we want to find A x B. In sequential code, you will need to have the following code.

In [0]:
import numpy as np
import pyopencl as cl
import time
a_np = np.random.randint(0, high=10, size=[100, 100])
b_np = np.random.randint(0, high=10, size=[100, 100])
result = np.zeros_like(a_np)
for i in range(len(a_np)):
  for j in range(len(b_np[0])):
    for k in range(len(b_np)):
       result[i][j] += a_np[i][k] * b_np[k][j]
print(result)
print(np.matmul(a_np,b_np))
print(np.linalg.norm(result - np.matmul(a_np, b_np)))


[[2074 1823 1926 ... 2136 1999 1891]
 [2208 1893 2017 ... 2027 1894 2080]
 [2334 1944 1834 ... 1968 1867 1967]
 ...
 [2105 1864 1837 ... 2085 1890 1846]
 [2412 1960 2270 ... 2213 2040 1981]
 [2012 1873 1954 ... 2214 1943 1960]]
[[2074 1823 1926 ... 2136 1999 1891]
 [2208 1893 2017 ... 2027 1894 2080]
 [2334 1944 1834 ... 1968 1867 1967]
 ...
 [2105 1864 1837 ... 2085 1890 1846]
 [2412 1960 2270 ... 2213 2040 1981]
 [2012 1873 1954 ... 2214 1943 1960]]
0.0


# Question #2
Complete the OpenCL code below to perform matrix multiplcation, you can use pseducode from the slide (Parallal Algorithm). If your code is correct, the value of vector norm must be 0.0.

In [52]:
import numpy as np
import pyopencl as cl
import time
a_np = np.random.randint(0, high=10, size=[300, 300])
b_np = np.random.randint(0, high=10, size=[300, 300])

def perform_sequential_multiplication(a_np, b_np):
  result = np.zeros_like(a_np)
  for i in range(len(a_np)):
    for j in range(len(b_np[0])):
      for k in range(len(b_np)):
        result[i][j] += a_np[i][k] * b_np[k][j]
  return result

def perform_opencl_multiplication(a_np, b_np):
  res_np = np.zeros_like(a_np)
  # Your code here
  # New_a = a_np.flatten()
  # Naw_b = b_np.flatten()
  ctx = cl.create_some_context()
  queue = cl.CommandQueue(ctx)

  mf = cl.mem_flags
  a_g = cl.Buffer(ctx, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf=a_np)
  b_g = cl.Buffer(ctx, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf=b_np)
  prg = cl.Program(ctx, """
  __kernel void multipler(
      __global const int *a_g, __global const int *b_g, __global int *res_g)
    {
      int N = 300;
      int col = (get_global_id(0)*2);
      int row = (get_global_id(1)*2);
      for(int i = 0;i< N*2; i+=2 ){
        res_g[row*N+col] += a_g[row*N + i ] * b_g[i*N + col];
      }
 
    }
  """).build()

  res_g = cl.Buffer(ctx, mf.WRITE_ONLY, a_np.nbytes)
  prg.multipler(queue, a_np.shape, None, a_g, b_g, res_g)

  cl.enqueue_copy(queue, res_np, res_g)

  return res_np
print("array of a_np")
print(a_np)
print ("-----------------------------------------------------------------------------------------")
print("array of b_np")
print(b_np)
print ("-----------------------------------------------------------------------------------------")
start_time = time.time()
res_np = perform_sequential_multiplication(a_np, b_np)
end_time = time.time()
sequential_time = end_time - start_time;
print(res_np)
print ("-----------------------------------------------------------------------------------------")
print('Find vector norm to check result:', np.linalg.norm(res_np - np.matmul(a_np, b_np)))
print("Sequential processing time:", sequential_time)
print ("-----------------------------------------------------------------------------------------")
start_time = time.time()
res_np = perform_opencl_multiplication(a_np, b_np)
print(res_np)
end_time = time.time()
parallel_time = end_time - start_time
print ("-----------------------------------------------------------------------------------------")
print('Find vector norm to check result:', np.linalg.norm(res_np - np.matmul(a_np, b_np)))
print("Parallel processing time:", parallel_time)

print("Speed up:", sequential_time/parallel_time)



array of a_np
[[9 7 5 ... 4 7 9]
 [3 2 4 ... 4 1 7]
 [0 9 2 ... 9 7 7]
 ...
 [1 9 9 ... 2 5 3]
 [7 0 9 ... 8 8 3]
 [0 7 7 ... 4 6 2]]
-----------------------------------------------------------------------------------------
array of b_np
[[9 7 0 ... 8 1 3]
 [6 6 9 ... 1 5 2]
 [4 9 5 ... 6 9 9]
 ...
 [6 5 1 ... 7 7 9]
 [5 0 4 ... 3 7 4]
 [4 6 8 ... 6 5 1]]
-----------------------------------------------------------------------------------------
[[6598 6003 6552 ... 6427 6646 6630]
 [6423 5853 6307 ... 6498 5790 6190]
 [6134 5589 5937 ... 6161 5692 5780]
 ...
 [6684 5840 6174 ... 6446 6272 6148]
 [6716 5917 6402 ... 6683 6151 6245]
 [6809 6114 6695 ... 6681 6413 6495]]
-----------------------------------------------------------------------------------------
Find vector norm to check result: 0.0
Sequential processing time: 29.79215908050537
-----------------------------------------------------------------------------------------
[[6598 6003 6552 ... 6427 6646 6630]
 [6423 5853 6307 ... 64

# Question #3
Q3.1: When the input matrix size is 100 x 100, is the parallel code run faster?

Answer: Yes

Q3.2: If the parallel code runs faster, what is the speed up?

Answer:4.44

Q3. What if you increase the matrix size to 200x200, 300x300, 400x400 ... do you see the speed up increased or decreased? 

Answer:increased

Q4: Why?

Answer: Because small data with original process faster then small data with parallel process due to transfering data to/from GPU slower then original process.

Q6: Again, guess the ratio of paralle portion, please also show your calculation step.

Answer: from S = 1/1-P then move P to other side P = S-1/S
        then S = 184.63 then P = 0.994 so 
        the ratio of parallel portion is 99.4 %  